In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import Perceptron, SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import OneClassSVM
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from sklearn.decomposition import PCA

## for visualization
import plotly.express as px
import matplotlib
from datetime import datetime, date 
from datetime import timedelta
from matplotlib import pyplot as plt
import seaborn as sns

from datetime import datetime, date 
from datetime import timedelta
import time
import shap

import random
import pickle

In [ ]:
df_train = pd.read_csv("../input/bioresponse/train.csv")
df_test = pd.read_csv("../input/bioresponse/test.csv")
benchmark= pd.read_csv("../input/bioresponse/svm_benchmark.csv")

In [ ]:
df_train.Activity.value_counts()

In [ ]:
df_train.D4.value_counts()

In [ ]:
df_train.info()

In [ ]:
df_train.isna().sum().sort_values(ascending=False) ## not missing data 

In [ ]:
df_train.describe()

In [ ]:
#sns.pairplot(df_train.iloc[:,:10])

## linear dependance between : D6 & D9, D6 & D7 

In [ ]:
#df_train.corrwith(df_train.Activity)
#values = df_train.corrwith(df_train.Activity)[df_train.corrwith(df_train.Activity).values>0.05].index
#values

#first part : fit a basic RF/xgb without tuning or features selection etc.. 
#to have a first log loss value to improve

#seed = random.seed(10)

#scaler = StandardScaler()
train = df_train.drop('Activity', axis=1)
target = df_train.Activity

#train = scaler.fit_transform(train)
X_train, X_test,y_train, y_test = train_test_split(train,target, test_size=0.3)
rf = RandomForestClassifier()
xgb = XGBClassifier()


rf.fit(X_train,y_train)
pred_rf = rf.predict(X_test)
score_rf = accuracy_score(pred_rf,y_test)
loss_rf = log_loss(pred_rf,y_test)

xgb.fit(X_train,y_train)
pred_xgb = xgb.predict(X_test)

score_xgb = accuracy_score(pred_xgb, y_test)
loss_xgb = log_loss(pred_xgb, y_test)

#rf.feature_importances_

print(loss_rf, loss_xgb)
print(score_rf, score_xgb)

In [ ]:
## let's try removing features with really low correlation with the target 

#df = df_train.copy()
#df = df.loc[:,(df.corrwith(target)>0.05)]

#scaler = StandardScaler()
#train1 = df.drop('Activity', axis=1)
#target1 = df.Activity

#train1 = scaler.fit_transform(train1)
#X_train1, X_test1,y_train1, y_test1 = train_test_split(train1,target1, test_size=0.3)
#rf1 = RandomForestClassifier()
#xgb1 = XGBClassifier()

#rf1.fit(X_train1,y_train1)
#pred_rf1 = rf1.predict(X_test1)
#score_rf1 = accuracy_score(pred_rf1,y_test1)
#loss_rf1 = log_loss(pred_rf1,y_test1)

#xgb1.fit(X_train1,y_train1)
#pred_xgb1 = xgb1.predict(X_test1)
#loss_xgb1 = log_loss(pred_xgb1, y_test1)
#score_xgb1 = accuracy_score(y_test,pred_xgb1)
#rf.feature_importances_

#print(loss_rf1, loss_xgb1)
#print(score_xgb1)

In [ ]:
##### trying some stuff : like features importance and shapley values for the PCA approach notebook 

rf = RandomForestClassifier()
train = df_train.drop('Activity', axis=1)
y_train = df_train.Activity 
X_train = df_train.drop('Activity', axis=1)
X_test = df_test
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rf.fit(X_train,y_train)

rf.feature_importances_

In [ ]:
def plot_feature_importance(importance,names,model_type):

    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    plt.bar(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    plt.show()

plot_feature_importance(rf.feature_importances_,train.columns,'RANDOM FOREST')

In [ ]:
!pip install shap 

In [ ]:
#### SHAPLEY VALUES 

shap_values = shap.TreeExplainer(rf).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type='bar')

In [ ]:
sample = rf.predict_proba(df_test)
#submission = pd.DataFrame({"id": df_test.index, "target": sample})
#submission.drop('id', axis=1, inplace=True) 
#submission.to_csv("submission.csv", index=False)

submission = pd.DataFrame({'MoleculeId': np.arange(1, len(sample)+1),
                        'PredictedProbability': sample[:, 0]})
submission.to_csv('submission.csv', index=False)

In [ ]:
submission